In [318]:
import pandas as pd

In [319]:
df = pd.read_csv("datatset_consumer_complaints.csv")

In [320]:
df.head(2)

,ID,Company,Product,Issue,State,Submitted via,Date received,Date resolved,Timely response?,Consumer disputed?,state name,Date received.1,Date resolved.1,Resolution time(in days),Year,QTR (US FLY)
0,1615767,PHH Mortgage,Mortgage,"Loan servicing, payments, escrow account",FL,Web,10/20/2015,10/20/2015,Yes,No,Florida,10/20/2015,10/20/2015,0,2015,Q4
1,654223,Ocwen,Mortgage,"Loan servicing, payments, escrow account",NC,Web,3/1/2014,3/1/2014,Yes,No,North Carolina,1/3/2014,1/3/2014,0,2014,Q1


In [321]:
columns = list(df.columns)
columns.remove("Date received.1")
columns.remove("Date resolved.1")
columns.remove("State")

In [322]:
df = df.loc [ : , columns ]

In [323]:
df.columns

Index([' ID', 'Company', 'Product', 'Issue', 'Submitted via', 'Date received',
       'Date resolved', 'Timely response?', 'Consumer disputed?', 'state name',
       'Resolution time(in days)', 'Year', 'QTR (US FLY)'],
      dtype='object')

In [324]:
df.columns = ["id", "company", "product", "issue", "submitted_via",
       "date_received", "date_resolved", "timely_response",
       "consumer_disputed", "state",
       "resolution_time", "year", "qtr"]

In [325]:
df["year"] = df["year"].astype("str")

In [326]:
cat_cols = [ c for c in df.columns if df[c].dtypes == "O" ]
num_cols = [ c for c in df.columns if df[c].dtypes in ["int64", "float64"] ]

In [327]:
num_cols

['id', 'resolution_time']

In [328]:
cat_cols

['company',
 'product',
 'issue',
 'submitted_via',
 'date_received',
 'date_resolved',
 'timely_response',
 'consumer_disputed',
 'state',
 'year',
 'qtr']

In [329]:
group_cols = ["product", "submitted_via", "state", "year"]
group_cols = ["product", "submitted_via", "state", "issue", "year"]
# group_cols = ["product", "state", "year"]
# group_cols = ["year", "state"]
# group_cols = ["state_name"]
group_cols = cat_cols

In [330]:
prob_freq = df.groupby( group_cols, as_index = False ).agg({
    "id": ["count"]
})

In [331]:
prob_freq.columns = group_cols + ["complaints"]

In [332]:
prob_freq["complaints"] = prob_freq["complaints"] / prob_freq.shape[0]

In [333]:
prob_freq

,company,product,issue,submitted_via,date_received,date_resolved,timely_response,consumer_disputed,state,year,qtr,complaints
0,21st Mortgage Corporation,Mortgage,"Loan servicing, payments, escrow account",Email,02/18/2015,02/25/2015,Yes,No,Florida,2015,Q1,0.000072
1,21st Mortgage Corporation,Mortgage,"Loan servicing, payments, escrow account",Web,04/13/2015,04/13/2015,Yes,No,Michigan,2015,Q2,0.000072
2,21st Mortgage Corporation,Mortgage,"Loan servicing, payments, escrow account",Web,04/24/2015,04/24/2015,Yes,Yes,California,2015,Q2,0.000072
3,21st Mortgage Corporation,Mortgage,"Loan servicing, payments, escrow account",Web,11/25/2014,11/25/2014,Yes,Yes,Texas,2014,Q4,0.000072
4,21st Mortgage Corporation,Mortgage,"Loan servicing, payments, escrow account",Web,12/4/2016,12/4/2016,Yes,Yes,California,2016,Q2,0.000072
...,...,...,...,...,...,...,...,...,...,...,...,...
13816,Zwicker & Associates,Debt collection,Improper contact or sharing of info,Web,5/5/2016,06/17/2016,Yes,No,Maryland,2016,Q2,0.000072
13817,Zwicker & Associates,Debt collection,"Loan modification,collection,foreclosure",Phone,08/14/2013,08/15/2013,Yes,Yes,Maryland,2013,Q3,0.000072
13818,Zwicker & Associates,Debt collection,Taking/threatening an illegal action,Web,11/25/2014,11/25/2014,Yes,Yes,California,2014,Q4,0.000072
13819,Zwicker & Associates,Student loan,Dealing with my lender or servicer,Web,10/15/2016,10/19/2016,Yes,Yes,South Carolina,2016,Q4,0.000072


In [335]:
df.groupby( cat_cols , as_index=False ).agg({
    'resolution_time': [ 
        ( 'lower_bound' , lambda x: x.quantile(.25) - 1.5*(x.quantile(.75) - x.quantile(.25))  )
        , ( 'upper_bound' , lambda x: x.quantile(.75) + 1.5*(x.quantile(.75) - x.quantile(.25))  )
    ]
}).head(2)

KeyboardInterrupt: 